###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [19]:
import pandas as pd

###Archivo v75m-npi8.csv

Contiene información sobre el número de infraestructuras para la producción de drogas ilícitas destruidas. Entiéndase por infraestructuras para la producción de drogas aquellos laboratorios donde se utilizan sustancias químicas y equipos necesarios para preparar una sustancia controlada a partir de sus precursores. Estas infraestructuras se clasifican de acuerdo con el proceso de fabricación o de acuerdo con el tipo de producto fabricado.

In [20]:
df = pd.read_csv("https://www.datos.gov.co/resource/v75m-npi8.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad,unidad
0,2010-01-02T00:00:00.000,52.0,NARIÑO,52490,OLAYA HERRERA,1,UNIDAD
1,2010-01-03T00:00:00.000,5.0,ANTIOQUIA,05040,ANORI,1,UNIDAD
2,2010-01-03T00:00:00.000,25.0,CUNDINAMARCA,25572,PUERTO SALGAR,1,UNIDAD
3,2010-01-03T00:00:00.000,52.0,NARIÑO,52835,SAN ANDRES DE TUMACO,2,UNIDAD
4,2010-01-03T00:00:00.000,52.0,NARIÑO,52490,OLAYA HERRERA,1,UNIDAD


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32484 entries, 0 to 32483
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   fecha_hecho   32484 non-null  object 
 1   cod_depto     32483 non-null  float64
 2   departamento  32484 non-null  object 
 3   cod_muni      32483 non-null  object 
 4   municipio     32484 non-null  object 
 5   cantidad      32484 non-null  int64  
 6   unidad        32484 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.7+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [22]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad']
destroyed_labs_MinDefensa = df[relevant_cols]
destroyed_labs_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32484 entries, 0 to 32483
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_muni      32483 non-null  object
 1   departamento  32484 non-null  object
 2   municipio     32484 non-null  object
 3   fecha_hecho   32484 non-null  object
 4   cantidad      32484 non-null  int64 
 5   unidad        32484 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


### Convertir columna fecha_hecho a tipo date

In [23]:
# Hacer una copia explícita del DataFrame
destroyed_labs_MinDefensa_ = destroyed_labs_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
destroyed_labs_MinDefensa_['fecha_hecho'] = pd.to_datetime(destroyed_labs_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
destroyed_labs_MinDefensa_['fecha_hecho'] = destroyed_labs_MinDefensa_['fecha_hecho'].dt.year

In [24]:
destroyed_labs_MinDefensa_['fecha_hecho'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024], dtype=int32)

### Verificar valores nulos

In [25]:
destroyed_labs_MinDefensa_.isnull().sum()

,0
cod_muni,1
departamento,0
municipio,0
fecha_hecho,0
cantidad,0
unidad,0


Nota: El nulo más adelante lo abordamos

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [26]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(destroyed_labs_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['NARIÑO' 'ANTIOQUIA' 'CUNDINAMARCA' 'GUAVIARE' 'CAQUETA' 'CAUCA'
 'PUTUMAYO' 'SANTANDER' 'META' 'MAGDALENA' 'CALDAS' 'NORTE DE SANTANDER'
 'BOLIVAR' 'VALLE DEL CAUCA' 'CORDOBA' 'VICHADA' 'VAUPES' 'CHOCO'
 'AMAZONAS' 'BOYACA' 'RISARALDA' 'ARAUCA' 'TOLIMA' 'GUAINIA' 'CESAR'
 'HUILA' 'BOGOTA D.C.' 'ATLANTICO' 'CASANARE' 'LA GUAJIRA'
 'SIN ESTABLECER' 'SUCRE' 'VENEZUELA' 'PERU']

Categorías en la columna 'municipio':
['OLAYA HERRERA' 'ANORI' 'PUERTO SALGAR' 'SAN ANDRES DE TUMACO'
 'SAN JOSE DEL GUAVIARE' 'YARUMAL' 'PUERTO RICO' 'FLORENCIA' 'ITUANGO'
 'LEGUIZAMO' 'PUERTO ASIS' 'ORITO' 'YOLOMBO' 'EL PLAYON'
 'CARTAGENA DEL CHAIRA' 'BRICEÑO' 'LINARES' 'SANTA MARTA' 'VILLAGARZON'
 'EL RETORNO' 'TARAZA' 'PUERTO GAITAN' 'SAMANA' 'CIENAGA' 'LA TOLA'
 'VALDIVIA' 'EL TARRA' 'SAN CALIXTO' 'ALTOS DEL ROSARIO'
 'SAN JUAN DE ARAMA' 'EL DONCELLO' 'ARGELIA' 'TIBU' 'BOLIVAR'
 'PUERTO CAICEDO' 'FLORIDA' 'PAEZ' 'CALAMAR' 'SARDINATA' 'CUMBITARA'
 'MAPIRIPAN' 'TIERRAL

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [27]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [28]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    destroyed_labs_MinDefensa_[col] = destroyed_labs_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [29]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df destroyed_labs_MinDefensa_

In [30]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
destroyed_labs_MinDefensa_['cod_muni'] = destroyed_labs_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = destroyed_labs_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [31]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 3 7]


In [32]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,32480,0.999877
1,7,3,0.000092
2,3,1,0.000031


In [33]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = destroyed_labs_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    52490
1    05040
2    25572
3    52835
4    52490

Muestra de registros con longitud 7:
      cod_muni
29655  6040101
30195  6040101
30371  6040101

Muestra de registros con longitud 3:
     cod_muni
2196      nan



In [34]:
# Verificar longitudes de 'cod_mupi' igual a 3 y 7 dígitos
cod_muni_3_6_7 = destroyed_labs_MinDefensa_[
    (destroyed_labs_MinDefensa_['cod_muni'].apply(len) == 3) |
    (destroyed_labs_MinDefensa_['cod_muni'].apply(len) == 7)
]
cod_muni_3_6_7['departamento'].value_counts()

,count
departamento,
PERU,3
SIN ESTABLECER,1


Nota: Como se aprecia los 'cod_mun' con longitudes diferentes a 5 dígitos, que en total suman 4 registros de un total de 32.488, corresponden a laboratorios en el exterios y uno 'SIN ESTABLECER'. Procedemos a excluirlas porque para efectos del proyecto no son importantes

In [35]:
destroyed_labs_MinDefensa_ = destroyed_labs_MinDefensa_[destroyed_labs_MinDefensa_['cod_muni'].apply(len) == 5]

Nota: El 100%  de los codigos que identifican a los municipios en el df destroyed_labs_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [36]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [39]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [40]:
# Comparar listas de departamento
compare_lists(destroyed_labs_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en destroyed_labs_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en destroyed_labs_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'VENEZUELA', 'PERU'}


  Nota: Aún figuran en el df incautaciones hechas en el exterior: 'VENEZUELA', 'PERU'.

In [41]:
# Verificar cuantos registros figuran a nombre de 'VENEZUELA', 'PERU''
confiscation_overseas = destroyed_labs_MinDefensa_[destroyed_labs_MinDefensa_['departamento'].isin(['VENEZUELA', 'PERU'])]
confiscation_overseas['departamento'].value_counts()

,count
departamento,
VENEZUELA,6
PERU,2


In [42]:
# Se procede a eliminar esos registros
destroyed_labs_MinDefensa_ = destroyed_labs_MinDefensa_[~(destroyed_labs_MinDefensa_['departamento'].isin(['VENEZUELA', 'PERU']))]

In [43]:
# Comparar nuevamente las listas de departamento
compare_lists(destroyed_labs_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en destroyed_labs_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en destroyed_labs_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [44]:
# Comparar listas de códigos
compare_lists(destroyed_labs_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en destroyed_labs_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en destroyed_labs_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [45]:
destroyed_labs_MinDefensa_[destroyed_labs_MinDefensa_['cod_muni'] == '94663']

,cod_muni,departamento,municipio,fecha_hecho,cantidad,unidad
8377,94663,GUAINIA,MAPIRIPANA,2015,2,UNIDAD
9687,94663,GUAINIA,MAPIRIPANA,2016,2,UNIDAD


In [46]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name
1092,94001,GUAINIA,INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL
1098,94887,GUAINIA,PANA PANA
1099,94888,GUAINIA,MORICHAL


In [47]:
# Como no hay ningun en Guania de nombre MAPIRIAPANA eliminamos el registro
destroyed_labs_MinDefensa_ = destroyed_labs_MinDefensa_[~(destroyed_labs_MinDefensa_['cod_muni']== '94663')]

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [48]:
# Comparar listas de códigos
compare_lists(destroyed_labs_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en destroyed_labs_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en destroyed_labs_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [49]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_destroyed_labs_MinDefensa = destroyed_labs_MinDefensa_.drop(columns=columns_to_drop)

In [56]:
# Adicionar columna para trazabilidad de la fuente
final_destroyed_labs_MinDefensa['source_id'] = 33

In [57]:
final_destroyed_labs_MinDefensa.columns

Index(['dane_code', 'year_of_incident', 'amount', 'unit', 'source_id'], dtype='object')

In [58]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_destroyed_labs_MinDefensa.rename(columns=translation_map, inplace=True)

In [59]:
#Estructura final del dataset a integrar a la base de datos
final_destroyed_labs_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32470 entries, 0 to 32483
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         32470 non-null  object
 1   year_of_incident  32470 non-null  int32 
 2   amount            32470 non-null  int64 
 3   unit              32470 non-null  object
 4   source_id         32470 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 1.4+ MB


## Salvar en archivo csv en el drive

In [60]:
# Guardar en archivos CSV en el drive
final_destroyed_labs_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/destroyed_labs_MinDefensa.csv', index=False)